In [1]:
# Line magic functions that will allow for imports to be reloaded and not cached
%load_ext autoreload
%autoreload 2

# Imports
import os
import pandas as pd
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
from datetime import date

# Local
from scripts.extractmd import Extractor
from scripts.vectorindex import QnAEngine
from scripts.utilities import get_prompt_dict, get_questions, get_answers, get_procurement_content, get_config_data
from scripts.gen_results import gen_results

/home/rud/CFLA/vpp-cfla/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Global config**

In [2]:
embedding_conf = {
    "embeddingmodel": "BAAI/bge-m3",  # "BAAI/bge-m3" "nomic-ai/nomic-embed-text-v2-moe" # "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    "chunk_size": 1536,
    "chunk_overlap": 0,
    "top_similar": 5,
    "n4rerank": 0, #How many nodes to retrieve for reranking. If 0, reranker is not used
    "use_similar_chunks": True, #To use similar chunks or the whole document as the context
    "prevnext": True #to include in the context also the previouse and the next chunk of the current similar chunk
}
embedding=HuggingFaceEmbedding(model_name=embedding_conf["embeddingmodel"],trust_remote_code=True)

#For nomic-embed-text-v2-moe
#embedding=HuggingFaceEmbedding(model_name=embedding_conf["embeddingmodel"],trust_remote_code=True,query_instruction="search_query: ",text_instruction="search_document: ")

In [10]:
# LLM Setup 
llmmodelAzure = { "model": "gpt-4o",
                "version":os.environ.get('AZURE_OPENAI_VERSION',''),
                "azure_deployment":"gpt-4o",
                "azure_endpoint":os.environ.get('AZURE_ENDPOINT',''),
                "api_key":os.environ.get('AZURE_OPENAI_KEY','')}

llm=AzureOpenAI(azure_deployment=llmmodelAzure["azure_deployment"],
                azure_endpoint=llmmodelAzure["azure_endpoint"],temperature=0.0,
                api_version=llmmodelAzure["version"], api_key=llmmodelAzure["api_key"],
                timeout=120,max_retries=3,top_p=0.0001)
print(llmmodelAzure)

{'model': 'gpt-4o', 'version': '', 'azure_deployment': 'gpt-4o', 'azure_endpoint': '', 'api_key': ''}


In [4]:
extractor = Extractor() # Markdown doc extractor

In [5]:
#Ollama model
# llmmodelOllama = { "model": "gemma3:27b",
#                 "url":os.environ.get('OLLAMA_ENDPOINT',''),
#                 "context_window":"128000"}

#from llama_index.llms.ollama import Ollama
#llm = Ollama(base_url=llmmodelOllama["url"],
#             model=llmmodelOllama["model"], 
#             context_window=int(llmmodelOllama["context_window"]),
#            request_timeout=300.0,
#            temperature=0.0,
#            additional_kwargs={"seed":1337})

**PROCUREMENT FILE SETTINGS**

In [6]:
# Script dir for getting relative paths for notebook file
script_dir = globals()['_dh'][0] 

# Document paths
question_file_path = script_dir / "questions" / "questions.yaml"
prompt_file = script_dir / "questions" / "prompts.tsv"
report_dir = script_dir / "reports"
config_dir = script_dir / "demo_config" # "dev_config" # "config"
procurement_file_dir = script_dir / "demo" # "cfla_files"
answer_file_dir = script_dir / "answers"

# Loading static information TODO
question_dictionary = get_questions(question_file_path)
promptdict = get_prompt_dict(prompt_file)

Questions loaded


**MAIN Q/A GENERATION SCRIPT**

In [7]:
# TODO config file loop; For each config in config_folder_path; # TODO add parallel prompting
# Create an array of ini files and then call them in a loop; This will give us more control over which files to call/exclude

ini_files = [f for f in os.listdir(config_dir) if f.endswith('.ini')]
print(f"Found {len(ini_files)} config files in {config_dir}")

# TODO add a little indicator how many files are to be processes; + aproximate time to finish each file
for file in ini_files:
        configfile = config_dir / file
        print(f"Processing config file: {configfile}")
        procurement_id, procurement_file, agreement_file, answer_file = get_config_data(configfile, procurement_file_dir, answer_file_dir)
        answer_dictionary = get_answers(answer_file)

        # Getting markdown text from procurement doc
        procurement_content = get_procurement_content(extractor, procurement_file, agreement_file)
    
        # Creating FAISS vector index for the procurement document
        qnaengine = QnAEngine(embedding,llm)
        await qnaengine.createIndex(
                procurement_content,
                "Procurement",
                chunk_size=embedding_conf["chunk_size"],
                chunk_overlap=embedding_conf["chunk_overlap"]
                )

        # Generating results
        results_table = gen_results(qnaengine, configfile, embedding_conf, question_dictionary, answer_dictionary, promptdict)

        # Save output

        # TODO probably should merge the output file together; One output file for the entire run, that contains each specific procurement results
        # TODO maybe implement some crash prevention strategy, so if some file breaks it doesn't break the entire run
        data = pd.DataFrame(results_table, columns=["Nr", "Atbilde", "Sagaidāmā atbilde", "Pamatojums"])
        precision = (data['Atbilde'] == data['Sagaidāmā atbilde']).sum()/len(data)
        print(f"PRECIZITĀTE: {precision*100}%")

        # Save final output file
        with open(f"{report_dir}\{date.today():%d.%m}_{procurement_id.replace('/','_')}.htm", 'w', encoding='utf-8') as ofile:
                print(data.to_html(index=False).replace('\\n','<br>'),file=ofile)
                print(f"PRECIZITĀTE: {precision*100}%",file=ofile)

Found 1 config files in /home/rud/CFLA/vpp-cfla/demo_config
Processing config file: /home/rud/CFLA/vpp-cfla/demo_config/GNP202565.ini


<string>:38: SyntaxWarning: invalid escape sequence '\{'
<>:38: SyntaxWarning: invalid escape sequence '\{'
/tmp/ipykernel_16524/3942380688.py:38: SyntaxWarning: invalid escape sequence '\{'
  with open(f"{report_dir}\{date.today():%d.%m}_{procurement_id.replace('/','_')}.htm", 'w', encoding='utf-8') as ofile:
Generating embeddings: 100%|██████████ [ time left: 00:00 ]


19 segments created and vectorized.
Index is ready.
2 An exception in askQuestion: ValueError You must set an `api_key` parameter. Alternatively, you can set the AZURE_OPENAI_API_KEY env var OR set `use_azure_ad=True`.
An exception in askQuestion: ValueError You must set an `api_key` parameter. Alternatively, you can set the AZURE_OPENAI_API_KEY env var OR set `use_azure_ad=True`.
4 An exception in askQuestion: ValueError You must set an `api_key` parameter. Alternatively, you can set the AZURE_OPENAI_API_KEY env var OR set `use_azure_ad=True`.
An exception in askQuestion: ValueError You must set an `api_key` parameter. Alternatively, you can set the AZURE_OPENAI_API_KEY env var OR set `use_azure_ad=True`.
6 An exception in askQuestion: ValueError You must set an `api_key` parameter. Alternatively, you can set the AZURE_OPENAI_API_KEY env var OR set `use_azure_ad=True`.
An exception in askQuestion: ValueError You must set an `api_key` parameter. Alternatively, you can set the AZURE_OPE

Table for Questions

In [8]:


# ── Containers ──
all_correctness = {}   # filename → { Nr_str: 0|1, … }
master_qids = []       # list of all Nr’s in the order first seen

# ── Reloop, but capture correctness per Nr ──
for file in ini_files:
    

    per_file = {}
    for nr, grp in data.groupby("Nr"):
        nr_str = str(nr)                # e.g. "4", "9.1", "9.2"
        if nr_str not in master_qids:
            master_qids.append(nr_str)

        # 1 only if every row in grp matches
        is_correct = (grp["Atbilde"] == grp["Sagaidāmā atbilde"]).all()
        per_file[nr_str] = int(is_correct)

    all_correctness[file] = per_file

# ── Build the matrix ──
df = pd.DataFrame.from_dict(all_correctness, orient="index")  # rows=files, cols=Nr
df = df.T                                                 
df = df.reindex(master_qids)                               
df.index.name = "Question Nr"


print("Question correctness (1=all sub-rows(q0 and q) correct, 0=any wrong):")
print(df)



Question correctness (1=all sub-rows(q0 and q) correct, 0=any wrong):
             GNP202565.ini
Question Nr               
10                       1
15                       1
16                       1
17                       1
17-0                     1
...                    ...
7-0                      1
9-0                      1
9.1                      1
9.2                      1
9.3                      1

[177 rows x 1 columns]


Correctness % and plot

In [9]:

import matplotlib.pyplot as plt

# ── 1) Performance metrics ──
# df: rows=Question Nr, cols=filenames, values=0/1
correct_counts = df.sum(axis=1)                   # how many files got each question right
num_files     = df.shape[1]                       # total number of files
percentages   = (correct_counts / num_files) * 100

perf_df = pd.DataFrame({
    "CorrectCount": correct_counts,
    "Percentage":  percentages.round(1)
})

# ── 2) Display stats ──
print("Per‐question performance:")
print(perf_df)

# ── 3) Bar chart ──
plt.figure()
plt.bar(perf_df.index.astype(str), perf_df["CorrectCount"])
plt.xlabel("Question Nr")
plt.ylabel("Number of Correct Answers")
plt.ylim(0, num_files)
plt.xticks(rotation=90)
plt.title("Quesiton correctnes")
plt.tight_layout()
plt.show()


ModuleNotFoundError: No module named 'matplotlib'